# Machine Learining: Regression

In [15]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import t as t_dis
from scipy.stats import f as f_dis
from termcolor import colored
from tabulate import tabulate
from mpl_toolkits import mplot3d

## Simple Linear Regression
**Idea**\
We want to find a linear relation between $X$ en $Y$:
$$
Y = \beta_0 + \beta_1X.
$$
Setting
$
\hat{\beta}_0 = b_0$ and $ \hat{\beta}_1 = b_1,$ we become as estimate for $y$:
$$
\hat{y} = b_0 + b_1x + \epsilon.
$$

**Determing $b_0$ en $b_1$ using LSM**\
Given the Sum of the Square Errors
$$
\text{SSE} = \sum_{i=1}^n(y_i - \hat{y}_i)^2  = \sum_{i=1}^n(y_i - b_0 - b_1x_i)^2.
$$
Making $\text{SSE}$ minimal with respect to $b_0$ and $b_1$:
$$
\frac{\partial\text{SSE}}{\partial b_0} = 0,\quad \quad \frac{\partial\text{SSE}}{\partial b_1} = 0,  
$$
gives 
$$
b_1 = \frac{\overline{xy} - \overline{x}\ \overline{y}}{\overline{x^2} - \overline{x}^2}= \frac{\sum_{i=1}^n(x_i-\overline{x})(y_i - \overline{y})}{\sum_{i=1}^n(x_i-\overline{x})^2}, \quad \quad
b_0 = \overline{y} - b_1\overline{x} =\frac{\overline{x^2}\overline{y} - \overline{x} \ \overline{xy}}{\overline{x^2} - \overline{x}^2}.
$$

**Example**

In [16]:
class LinearRegression:
    
    def __init__(self, x, y):
        #Data
        self.x, self.y = x, y
        
        #Sample size
        self.n = len(x)
    
        #Linear regression function
        self.x_a, self.y_a = np.mean(x), np.mean(y)
        self.b1 = np.sum((x - self.x_a)*(y - self.y_a))/np.sum((x-self.x_a)**2)
        self.b0 = self.y_a - self.b1*self.x_a
        self.f = lambda t: self.b0 + self.b1*t
        
            
        #Helping variables
        self.y_p = self.f(self.x)                         
        self.SST = np.sum((self.y - self.y_a)**2)         
        self.SSE = np.sum((self.y - self.y_p)**2)        
        self.SSR = np.sum((self.y_p - self.y_a)**2)
        self.s = np.sqrt(self.SSE/(self.n - 2))
        self.s_x = np.sqrt(np.sum((self.x - self.x_a)**2)/self.n)
        
        #Determination coefficiënt
        self.R = np.sqrt(1 - self.SSE/self.SST)
    
    def __call__(self, x, ci = False, alpha = 0.05):
        if ci == True:
            var_y = self.s**2*(1/self.n + (x - self.x_a)**2/(self.n*self.s_x**2))
            t0 = t_dis.ppf(1 - alpha/2, self.n - 2)
            dy = t0*np.sqrt(var_y)
            y_p = self.b0 + self.b1*x
            print(f'{1 - alpha:.3f} C.I. E(Y|x): [{y_p - dy:.2f}, {y_p + dy:.2f}]')
            
        return self.b0 + self.b1*x
    
    def analysis(self, alpha = 0.05): 
        stat = tabulate([['x', f'{self.x_a:.2f}', f'{self.x.std():.2f}'], ['y',  f'{self.y_a:.2f}', f'{self.y.std():.2f}']],headers = [colored('Type', attrs = ['bold']), colored('Mean', attrs = ['bold']), colored('Std. err.', attrs = ['bold'])])
        
        #Confidence interval beta_0
        t = t_dis.ppf(1 - alpha/2, self.n - 2) 
        s = np.sqrt(self.SSE/(self.n - 2))
        Sxx = np.sum((self.x - self.x_a)**2)
        db0 = s/np.sqrt(self.n*Sxx)*np.sqrt(np.sum(self.x**2))*t
  
        #Confidence interval of b1
        db1 = s/np.sqrt(Sxx)*t
        s_x = np.sqrt(np.sum((self.x - self.x_a)**2)/self.n)
        
        #Significance of b1
        var_b1 = s**2/(self.n*s_x**2)
        T_b1 = self.b1/np.sqrt(var_b1)
        p_b1 = 2*(1 - t_dis.cdf(abs(T_b1), self.n - 2))
        
        #Significance of regression model
        F = self.SSR/s**2
        p = 1 - f_dis.cdf(F, 1, self.n-2)
        
        table_coeff = tabulate([['b0', f'{self.b0:.2f}', f'[{self.b0-db0:.2f}, {self.b0+db0:.2f}]', '\ ', '\ '], ['b1', f'{self.b1:.2f}', f'[{self.b1-db1:.2f}, {self.b1+db1:.2f}]', f'{T_b1:.2f}', f'{p_b1:.2f}']], headers = [colored('Coeff', attrs = ['bold']), colored('Value', attrs = ['bold']), colored(f'{1 - alpha:.3f}% C.I.', attrs = ['bold']), colored('t', attrs = ['bold']), colored('p-value', attrs = ['bold'])])
        
        table_anova = tabulate([['Regression', f'{self.SSR:.2f}', 1, f'{self.SSR:.2f}', f'{F:.2f}', f'{p:.2f}'], ['Error', f'{self.SSE:.2f}', self.n-2, f'{s**2:.2f}', '', ''], ['Total', f'{self.SST:.2f}', self.n-1, '', '', '']],headers = [colored('Source of variation', attrs = ['bold']), colored('Sum of Squares', attrs = ['bold']), colored('df', attrs = ['bold']), colored('Mean Square', attrs = ['bold']), colored('f', attrs = ['bold']), colored('p-value', attrs = ['bold'])])

        #summary
        print('Statistics\n' + stat)
        print('\nCoefficients\n' + table_coeff)
        print('\nVariance-analysis\n' + table_anova)
        
    def RegPlot(self, alpha = 0.05):
        z = np.arange(np.min(self.x), np.max(self.x), 0.01)
        
        var_y = self.s**2*( 1/self.n + (z - self.x_a)**2/(self.n*self.s_x**2))
        t0 = t_dis.ppf(1 - alpha/2, self.n - 2)
        dy = t0*np.sqrt(var_y)
        
        fig = plt.figure(figsize = (8,4), dpi = 100)
        axes = fig.add_axes([0, 0, 1, 1])
        
        plt.title('Linear Regression', fontsize = 14)
        plt.xlabel('$X$')
        plt.ylabel('$Y$')
        
        Y = (lambda x: self.b0 + self.b1*x)(z)
        plt.plot(self.x, self.y, ls = '', markersize = 10, marker = 'o', color = 'blue', label = 'data')
        plt.plot(z, Y, lw = 3, color = 'red', ls = '-', label = 'regression line')
        plt.plot(z, Y-dy, lw = 1, color = 'red', ls = '--', label = f'{1 - alpha:.3f} C.I. $E(Y|x)$')
        plt.plot(z, Y+dy, lw = 1, color = 'red', ls = '--')
        plt.fill_between(z, (Y - dy), (Y + dy), color = 'r', alpha=.1)
    
        plt.grid()
        plt.legend()
        plt.show()
    
    def ResPlot(self):
        #Residuals
        r = self.y - self.y_p
        
        fig = plt.figure(figsize = (8, 4), dpi = 100)
        axes = fig.add_axes([0, 0, 1, 1])
        
        plt.title('Residual scatter plot', fontsize = 14)
        plt.ylabel('residuals $r$', fontsize = 14)
        plt.xlabel(r'Predictions $\hat{y}$', fontsize = 14)
        
        plt.plot(self.y_p, r, ls = '', markersize = 10, marker = 'o', color = 'blue')
        plt.axhline(y = 0, lw = 3, ls = '--', color = 'red')
        
        plt.grid()
        plt.show()

In [17]:
Y = np.array([25.5, 31.2, 25.9, 38.4, 18.4, 26.7, 26.4, 25.9, 32.0, 25.2, 39.7, 35.7, 26.5])

In [18]:
x1 = np.array([1.74, 6.32, 6.22, 10.52, 1.19, 1.22, 4.10, 6.32, 4.08, 4.15, 10.15, 1.72, 1.70])
x2 = np.array([5.30, 5.42, 8.41, 4.63, 11.60, 5.85, 6.62, 8.72, 4.42, 7.60, 4.83, 3.12, 5.30])
x3 = np.array([10.80, 9.40, 7.20, 8.50, 9.40, 9.90, 8.00, 9.10, 8.70, 9.20, 9.40, 7.60, 8.20])
X = np.zeros((len(x1), 4))
X[:, 0] = np.ones(len(x1))
X[:, 1] = x1
X[:, 2] = x2
X[:, 3] = x3

In [19]:
#l = LinearRegression(x1, Y)

In [20]:
#l.analysis()

In [21]:
#l.RegPlot(alpha = 0.05)

In [22]:
#l.ResPlot()

In [23]:
#print(l(6, ci = True))

## Multiple linear regression

In [24]:
class MultipleLinearRegression:
    
    def __init__(self, X, Y):
        #Sample size 
        self.n = len(Y)
        
        #Regression
        self.Y, self.X = Y, X
        self.C = np.linalg.inv(X.T@X)
        self.b = self.C@X.T@Y
        self.f = lambda x: x@self.b
    
        #Helping variables
        self.y_p = self.f(self.X)
        self.y_a = np.mean(self.Y)
        self.SST = np.sum((self.Y - self.y_a)**2)         
        self.SSE = np.sum((self.Y - self.y_p)**2)        
        self.SSR = np.sum((self.y_p - self.y_a)**2)
        self.s = np.sqrt(self.SSE/(self.n - len(self.b)))
    
    def __call__(self, x):
        x0 = np.array([1] + list(x))
        return x@self.b
    
    def analysis(self, alpha = 0.05):
        #Statistics
        st = [['x'+str(i), f'{np.mean(self.X[:,i]):.2f}', f'{self.X[:,i].std():.2f}'] for i in range(1, len(self.X[0]))]
        st_y = ['y',  f'{np.mean(self.Y):.2f}', f'{self.Y.std():.2f}']
        stat = tabulate([st_y] + st, headers = [colored('Type', attrs = ['bold']), colored('Mean', attrs = ['bold']), colored('Std. err.', attrs = ['bold'])])
        
        #Confidence intervals parameters
        T = self.b/(self.s*np.sqrt(np.diag(self.C)))
        t0 = t_dis.ppf(1 - alpha/2, self.n - len(self.b))
        dT = self.s*np.sqrt(np.diag(self.C))*t0
        p = 2*(1 - t_dis.cdf(abs(T), self.n - len(self.b)))
        cff = [[f'b'+str(i), self.b[i], f'[{T[i] - dT[i]:.2f}, {T[i] + dT[i]:.2f}]', f'{T[i]:.2f}', f'{p[i]:.2f}'] for i in range(len(T))]
        
        #Anova table
        
        
        
        #Output
        print('Statistics\n'+stat)
        print('\nCoefficients\n' + tabulate(cff, headers = [colored('Coeff', attrs = ['bold']), colored('Value', attrs = ['bold']), colored(f'{1 - alpha:.3f}% C.I.', attrs = ['bold']), colored('t', attrs = ['bold']), colored('p-value', attrs = ['bold'])]))

**Example**

In [25]:
#ml = MultipleLinearRegression(X, Y)

In [26]:
#print(ml.b)

In [14]:
#ml.analysis()